# Coronavirus Superlab
Date: 12-2-2024 * updated to remove -1 in Seqio and my proteins match. I also made it so it can read multiple sequences at once and i also parsed the translated region from the genbank file for a better cds dictionary.

## Set up Environment

In [21]:
import Bio
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord

## Task A 
### Cut out all gene subsequences from full genome and save in file
1. Obtain coronavirus genome sequence in two formats. FASTA and GENBANK. 
2. In a Jupyter notebook, write Python code that reads the GENBANK formatted coronavirus genome DNA sequence file, parses the coordinates of all CDS gene subsequences, and stores those values in a Python dictionary.
3. In a Jupyter notebook, write Python code that reads the FASTA formatted coronavirus genome DNA sequence file, cut out the actual sequences from the FASTA file using the dictionary from Task B (Step 1),  and write a text file that contains all of the coronavirus CDS DNA sequences in a single concatenated DNA file.

** Notes ** 
- All the gene sequences are now in the same file that can be _parsed_ using lines that start with the FASTA ‘>’ delimiter.
- A string of triplet codons in RNA is called the protein codon sequence (CDS) – See BioBytes.  If you look in the GENBANK file above the actual sequence
- The CDS DNA sequences are in the GenBank file, so you can check the accuracy of your sequence extraction. 


### Part 1

In [22]:
genbank_file = "MN908947.genbank"
gene_cds_dictionary = {}

# parse file

for record in SeqIO.parse (genbank_file, "genbank"):
    for feature in record.features:
        if feature.type =="CDS":
            # extract gene name
            gene_name = feature.qualifiers.get("gene", ["Unknown"]) [0]

            # extract cds coordinates
            start = feature.location.start
            end = feature.location.end
            strand = feature.location.strand
            #extract translated region
            translation = feature.qualifiers.get("translation", [""])[0]

            #save to dictionary
            gene_cds_dictionary[gene_name] = {
                "start": int(start),
                "end": int(end),
                "strand": strand,
                "translation": translation,
            }

print(gene_cds_dictionary)

{'orf1ab': {'start': 265, 'end': 21555, 'strand': 1, 'translation': 'MESLVPGFNEKTHVQLSLPVLQVRDVLVRGFGDSVEEVLSEARQHLKDGTCGLVEVEKGVLPQLEQPYVFIKRSDARTAPHGHVMVELVAELEGIQYGRSGETLGVLVPHVGEIPVAYRKVLLRKNGNKGAGGHSYGADLKSFDLGDELGTDPYEDFQENWNTKHSSGVTRELMRELNGGAYTRYVDNNFCGPDGYPLECIKDLLARAGKASCTLSEQLDFIDTKRGVYCCREHEHEIAWYTERSEKSYELQTPFEIKLAKKFDTFNGECPNFVFPLNSIIKTIQPRVEKKKLDGFMGRIRSVYPVASPNECNQMCLSTLMKCDHCGETSWQTGDFVKATCEFCGTENLTKEGATTCGYLPQNAVVKIYCPACHNSEVGPEHSLAEYHNESGLKTILRKGGRTIAFGGCVFSYVGCHNKCAYWVPRASANIGCNHTGVVGEGSEGLNDNLLEILQKEKVNINIVGDFKLNEEIAIILASFSASTSAFVETVKGLDYKAFKQIVESCGNFKVTKGKAKKGAWNIGEQKSILSPLYAFASEAARVVRSIFSRTLETAQNSVRVLQKAAITILDGISQYSLRLIDAMMFTSDLATNNLVVMAYITGGVVQLTSQWLTNIFGTVYEKLKPVLDWLEEKFKEGVEFLRDGWEIVKFISTCACEIVGGQIVTCAKEIKESVQTFFKLVNKFLALCADSIIIGGAKLKALNLGETFVTHSKGLYRKCVKSREETGLLMPLKAPKEIIFLEGETLPTEVLTEEVVLKTGDLQPLEQPTSEAVEAPLVGTPVCINGLMLLEIKDTEKYCALAPNMMVTNNTFTLKGGAPTKVTFGDDTVIEVQGYKSVNITFELDERIDKVLNEKCSAYTVELGTEVNEFACVVADAVIKTLQPVSELLTPLGIDLDEWSMATYYLFDESGEFKLASHMYCSFYPPDEDE

### Part 2 

In [23]:
# paste gene coordinates from part 1

gene_coordinates = {
    'orf1ab': {'start': 265, 'end': 21555}, 
    'S': {'start': 21562, 'end': 25384}, 
    'ORF3a': {'start': 25392, 'end': 26220}, 
    'E': {'start': 26244, 'end': 26472},
    'M': {'start': 26522, 'end': 27191}, 
    'ORF6': {'start': 27201, 'end': 27387}, 
    'ORF7a': {'start': 27393, 'end': 27759},
    'ORF8': {'start': 27893, 'end': 28259}, 
    'N': {'start': 28273, 'end': 29533},
    'ORF10': {'start': 29557, 'end': 29674}
}


# input files 

input_fasta = "MN908947.fasta" # enter FASTA file to extract genes from
output_fasta = "extracted_genes_MN908947.fasta" # name of extracted genes file that will be created
# Open the genome file and read it into memory
genome = SeqIO.read(input_fasta, "fasta")

with open(output_fasta, "w") as output_handle:
   # Loop through all genes in the coordinate dictionary
    for gene_name, coordinates in gene_coordinates.items():
        start = coordinates['start']
        end = coordinates['end']
        
        # Extract the gene sequence
        gene_sequence = genome.seq[start:end]  # -1 because FASTA is 1-indexed - going to change this by removing the -1 after start
        
        # Create a SeqRecord object
        gene_record = SeqRecord(gene_sequence, id=gene_name, description="")
        
        # Write the gene to the output file in FASTA format
        SeqIO.write(gene_record, output_handle, "fasta")

print(f"Extracted gene sequences have been written to {output_fasta}")

Extracted gene sequences have been written to extracted_genes_MN908947.fasta


In [24]:
def process_genbank_file(genbank_file, output_file):
    gene_cds_dictionary = {}
    translated_sequences = {}

    # parse file
    for record in SeqIO.parse(genbank_file, "genbank"):
        for feature in record.features:
            if feature.type == "CDS":
                # extract gene name
                gene_name = feature.qualifiers.get("gene", ["Unknown"])[0]

                # extract cds coordinates
                start = feature.location.start
                end = feature.location.end
                strand = feature.location.strand

                # extract translated region
                translation = feature.qualifiers.get("translation", [""])[0]

                # save to dictionary
                gene_cds_dictionary[gene_name] = {
                    "start": int(start),
                    "end": int(end),
                    "strand": strand,
                    "translation": translation,
                }

                # Save translated sequence
                if translation:
                    translated_sequences[gene_name] = translation

    # Write translated sequences to a FASTA file
    with open(output_file, "w") as f:
        for gene_name, sequence in translated_sequences.items():
            f.write(f">{gene_name}\n{sequence}\n")

    return gene_cds_dictionary


# Usage
genbank_file = "MN908947.genbank"
output_file = "translated_cds_genes.fasta"
gene_cds_dictionary = process_genbank_file(genbank_file, output_file)

print("Gene CDS Dictionary:")
print(gene_cds_dictionary)
print(f"\nTranslated CDS sequences have been written to {output_file}")


Gene CDS Dictionary:
{'orf1ab': {'start': 265, 'end': 21555, 'strand': 1, 'translation': 'MESLVPGFNEKTHVQLSLPVLQVRDVLVRGFGDSVEEVLSEARQHLKDGTCGLVEVEKGVLPQLEQPYVFIKRSDARTAPHGHVMVELVAELEGIQYGRSGETLGVLVPHVGEIPVAYRKVLLRKNGNKGAGGHSYGADLKSFDLGDELGTDPYEDFQENWNTKHSSGVTRELMRELNGGAYTRYVDNNFCGPDGYPLECIKDLLARAGKASCTLSEQLDFIDTKRGVYCCREHEHEIAWYTERSEKSYELQTPFEIKLAKKFDTFNGECPNFVFPLNSIIKTIQPRVEKKKLDGFMGRIRSVYPVASPNECNQMCLSTLMKCDHCGETSWQTGDFVKATCEFCGTENLTKEGATTCGYLPQNAVVKIYCPACHNSEVGPEHSLAEYHNESGLKTILRKGGRTIAFGGCVFSYVGCHNKCAYWVPRASANIGCNHTGVVGEGSEGLNDNLLEILQKEKVNINIVGDFKLNEEIAIILASFSASTSAFVETVKGLDYKAFKQIVESCGNFKVTKGKAKKGAWNIGEQKSILSPLYAFASEAARVVRSIFSRTLETAQNSVRVLQKAAITILDGISQYSLRLIDAMMFTSDLATNNLVVMAYITGGVVQLTSQWLTNIFGTVYEKLKPVLDWLEEKFKEGVEFLRDGWEIVKFISTCACEIVGGQIVTCAKEIKESVQTFFKLVNKFLALCADSIIIGGAKLKALNLGETFVTHSKGLYRKCVKSREETGLLMPLKAPKEIIFLEGETLPTEVLTEEVVLKTGDLQPLEQPTSEAVEAPLVGTPVCINGLMLLEIKDTEKYCALAPNMMVTNNTFTLKGGAPTKVTFGDDTVIEVQGYKSVNITFELDERIDKVLNEKCSAYTVELGTEVNEFACVVADAVIKTLQPVSELLTPLGIDLDEWSMATYYLFDE

## Task B
### Transcribe all the genes from DNA to RNA.

1. In a Jupyter notebook, write Python code that transcribes all DNA CDS sequences into single concatenated RNA sequence file in FASTA format.


In [25]:

# input files
input_file = "extracted_genes_MN908947.fasta"  # Replace with your input file name
output_file = "DNA_TO_RNA.fasta"  # Replace with your desired output file name

In [26]:

def transcribe_dna_to_rna(input_file, output_file):
    dna_to_rna = {
        "A": "A",
        "T": "U",
        "G": "G",
        "C": "C",
        "N": "N",
        "a": "A",
        "t": "U",
        "g": "G",
        "c": "C",
        "n": "N",
    }

    try:
        with open(input_file, "r") as infile, open(output_file, "w") as outfile:
            current_sequence = []
            current_header = ""
            sequence_count = 0

            for line in infile:
                line = line.strip()
                if line.startswith(">"):
                    # If we have a previous sequence, transcribe and write it
                    if current_sequence:
                        dna_sequence = "".join(current_sequence)
                        rna_sequence = "".join(
                            [
                                dna_to_rna.get(nucleotide, "")
                                for nucleotide in dna_sequence
                            ]
                        )
                        outfile.write(f"{current_header}\n{rna_sequence}\n")
                        sequence_count += 1

                    # Start a new sequence
                    current_header = line
                    current_sequence = []
                else:
                    current_sequence.append(line.upper())

            # Transcribe and write the last sequence
            if current_sequence:
                dna_sequence = "".join(current_sequence)
                rna_sequence = "".join(
                    [dna_to_rna.get(nucleotide, "") for nucleotide in dna_sequence]
                )
                outfile.write(f"{current_header}\n{rna_sequence}\n")
                sequence_count += 1

        print(
            f"Transcription complete. {sequence_count} RNA sequences written to {output_file}"
        )

    except FileNotFoundError:
        print(f"File {input_file} not found.")
        return

transcribe_dna_to_rna(input_file, output_file)


Transcription complete. 10 RNA sequences written to DNA_TO_RNA.fasta


## Task C
### Translate all of the genes from RNA into Proteins

**_Notes:_**

- The protein sequences are in the GenBank file, so you can check the accuracy of your sequence extraction. 
- Check out the CodeBytes section for help with the genetic code dictionary.
- Stop codons do not code for an amino acid.  The penultimate codon would be the last amino acid.

In [27]:


# Define the input and output file names
input_file = "DNA_TO_RNA.fasta"  # Replace with your actual input RNA file name
output_file = "translated_proteins.fasta"  # Replace with your desired output file name

# Standard RNA codon table
rna_codon_table = {
    'UUU': 'F', 'UUC': 'F', 'UUA': 'L', 'UUG': 'L',
    'UCU': 'S', 'UCC': 'S', 'UCA': 'S', 'UCG': 'S',
    'UAU': 'Y', 'UAC': 'Y', 'UAA': '*', 'UAG': '*',
    'UGU': 'C', 'UGC': 'C', 'UGA': '*', 'UGG': 'W',
    'CUU': 'L', 'CUC': 'L', 'CUA': 'L', 'CUG': 'L',
    'CCU': 'P', 'CCC': 'P', 'CCA': 'P', 'CCG': 'P',
    'CAU': 'H', 'CAC': 'H', 'CAA': 'Q', 'CAG': 'Q',
    'CGU': 'R', 'CGC': 'R', 'CGA': 'R', 'CGG': 'R',
    'AUU': 'I', 'AUC': 'I', 'AUA': 'I', 'AUG': 'M',
    'ACU': 'T', 'ACC': 'T', 'ACA': 'T', 'ACG': 'T',
    'AAU': 'N', 'AAC': 'N', 'AAA': 'K', 'AAG': 'K',
    'AGU': 'S', 'AGC': 'S', 'AGA': 'R', 'AGG': 'R',
    'GUU': 'V', 'GUC': 'V', 'GUA': 'V', 'GUG': 'V',
    'GCU': 'A', 'GCC': 'A', 'GCA': 'A', 'GCG': 'A',
    'GAU': 'D', 'GAC': 'D', 'GAA': 'E', 'GAG': 'E',
    'GGU': 'G', 'GGC': 'G', 'GGA': 'G', 'GGG': 'G'
}

def translate_rna(rna_sequence):
    """Translate RNA sequence to amino acid sequence"""
    protein = ''
    for i in range(0, len(rna_sequence), 3):
        codon = rna_sequence[i:i+3]
        if len(codon) == 3:
            amino_acid = rna_codon_table.get(codon, 'X')  # 'X' for unknown codons
            protein += amino_acid
    return protein
print(f"Starting translation process...")
print(f"Input file: {input_file}")
print(f"Output file: {output_file}")

# Read the input file and translate each sequence

with open(input_file, 'r') as infile, open(output_file, 'w') as outfile:
    current_sequence = []
    current_header = ""
    sequence_count = 0

    for line in infile:
        line = line.strip()
        if line.startswith('>'):
            # If we have a previous sequence, translate and write it
            if current_sequence:
                rna_sequence = ''.join(current_sequence)
                protein_sequence = translate_rna(rna_sequence)
                outfile.write(f"{current_header}\n{protein_sequence}\n")
                sequence_count += 1
            
            # Start a new sequence
            current_header = line
            current_sequence = []
        else:
            current_sequence.append(line)

    # Translate and write the last sequence
    if current_sequence:
        rna_sequence = ''.join(current_sequence)
        protein_sequence = translate_rna(rna_sequence)
        outfile.write(f"{current_header}\n{protein_sequence}\n")
        sequence_count += 1

print(f"Translation complete. {sequence_count} protein sequences written to {output_file}")

# Verify file contents
with open(output_file, 'r') as check_file:
    content = check_file.read()
    print(f"Output file size: {len(content)} characters")
    print(f"First 100 characters of output file: {content[:100]}")



Starting translation process...
Input file: DNA_TO_RNA.fasta
Output file: translated_proteins.fasta
Translation complete. 10 protein sequences written to translated_proteins.fasta
Output file size: 9773 characters
First 100 characters of output file: >orf1ab
MESLVPGFNEKTHVQLSLPVLQVRDVLVRGFGDSVEEVLSEARQHLKDGTCGLVEVEKGVLPQLEQPYVFIKRSDARTAPHGHVMVELVAEL
